# <a id='toc1_'></a>[__Карта головных филиалов Сбербанка__](#toc0_)

**Содержание**<a id='toc0_'></a>
- [__Карта головных филиалов Сбербанка__](#toc1_)
  - [__Импорты и настройки__](#toc1_1_)
  - [__Парсинг и геокодирование филиалов__](#toc1_2_)
  - [__Отрисовка филиалов на карте__](#toc1_3_)

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

***
## <a id='toc1_1_'></a>[__Импорты и настройки__](#toc0_)

In [1]:
# стандартная библиотека
import os
from time import sleep

In [2]:
# сторонние библиотеки
import bokeh.io, bokeh.models, bokeh.plotting
import pandas as pd
import pyproj
from bs4 import BeautifulSoup
from dadata import Dadata
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
pd.set_option("display.max_colwidth", 0)
pd.set_option("display.max_rows", 100)

***
## <a id='toc1_2_'></a>[__Парсинг и геокодирование филиалов__](#toc0_)

In [5]:
# https://cbr.ru/finorg/foinfo/branches/?id=1315037838265

with open("source.html", encoding="utf-8") as fh:
    soup = BeautifulSoup(fh, "html.parser")

In [6]:
# https://dadata.ru/api/geocode/

if not os.path.isfile("branches.xlsx"):
    dadata = Dadata(
        os.environ["DADATA_TOKEN"],
        os.environ["DADATA_SECRET"]
    )

    columns = [
        "ID",          # номер филиала
        "NAME",        # наименование филиала
        "ADDRESS",     # адрес
        "LAT",         # широта
        "LON",         # долгота
        "DATE_OPENED"  # дата открытия филиала
    ]

    records = []

    for row in soup.find_all("tr", class_="SubFilials"):
        cells = [cell.get_text() for cell in row.find_all("td")]
        del cells[2]  # столбец из слова "филиал"
        result = dadata.clean("address", cells[2])
        cells.insert(3, result["geo_lat"])
        cells.insert(4, result["geo_lon"])
        records.append(dict(zip(columns, cells)))
        sleep(2)  # не перегружать dadata

    df = pd.DataFrame.from_records(records)
    df.to_excel("branches.xlsx", index=False)

***
## <a id='toc1_3_'></a>[__Отрисовка филиалов на карте__](#toc0_)

In [7]:
df = pd.read_excel("branches.xlsx")
transformer = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3857")
df["x"], df["y"] = transformer.transform(df.LAT, df.LON)
df

,ID,NAME,ADDRESS,LAT,LON,DATE_OPENED,x,y
0,1662,Абаканское отделение № 8602,"655017, Республика Хакасия, г.Абакан, ул.Пушкина, д.165, строение 1, помещение 2Н",53.715185,91.433776,20.05.1997,1.017836e+07,7.116399e+06
1,7,Адыгейское отделение №8620,"385011, Республика Адыгея, г. Майкоп, ул. Димитрова, 4, корпус 2",44.616071,40.070127,20.05.1997,4.460586e+06,5.561281e+06
2,16,Алтайское отделение № 8644,"656038, Алтайский край, г. Барнаул, проспект Комсомольский, 106-А",53.349199,83.784801,20.05.1997,9.326881e+06,7.047853e+06
3,88,Архангельское отделение № 8637,"163000, Архангельская область, городской округ город Архангельск, наб. Северной Двины, д. 55",64.531091,40.527059,20.05.1997,4.511452e+06,9.485929e+06
4,112,Астраханское отделение N 8625,"414000, г. Астрахань, Кировский район, ул. Кирова, 41",46.343679,48.038117,20.05.1997,5.347579e+06,5.835596e+06
5,422,Байкальский банк,"664011, г. Иркутск, ул. Нижняя Набережная, 10",52.292966,104.287493,20.05.1997,1.160923e+07,6.853272e+06
6,130,Башкирское отделение № 8598,"450059, Республика Башкортостан, г.Уфа, ул.Рихарда Зорге, 5.",54.748510,55.982868,20.05.1997,6.231984e+06,7.313209e+06
7,188,Белгородское отделение №8592,"308000, Белгородская область, г.Белгород, Гражданский проспект, 52",50.595372,36.591575,20.05.1997,4.073355e+06,6.550029e+06
8,1643,Биробиджанское отделение №4157,"679016, Еврейская автономная область, г. Биробиджан, ул.Шолом-Алейхема, 16",48.791844,132.929326,20.05.1997,1.479762e+07,6.239615e+06
9,70,Благовещенское отделение №8636,"675000, Амурская область, г. Благовещенск, ул. Зейская 240",50.262171,127.516109,20.05.1997,1.419503e+07,6.491804e+06


In [ ]:
plot = bokeh.plotting.figure(
    title="Головные филиалы Сбербанка",
    height=600,
    width=1200,
    x_axis_type="mercator",
    y_axis_type="mercator",
    tools="pan,wheel_zoom,hover,reset,save",
    toolbar_location="above",
    active_scroll="wheel_zoom"
)

hover = plot.select_one(bokeh.models.HoverTool)
hover.point_policy = "follow_mouse"

hover.tooltips = f"""
<div>
    <div class="bokeh_hover_tooltip">Номер: @ID</div>
    <div class="bokeh_hover_tooltip">Наименование: @NAME</div>
    <div class="bokeh_hover_tooltip">Адрес: @ADDRESS</div>
</div>
"""

plot.add_tile("CARTODBPOSITRON_RETINA")

plot.scatter(
    x="x",
    y="y",
    source=df,
    size=10,
    color="orangered",
    marker="star"
)

bokeh.io.curdoc().theme = "dark_minimal"
# bokeh.plotting.show(plot)

bokeh.plotting.output_file("branches.html", mode="inline")
bokeh.plotting.save(plot)